# Ejercicios Prácticos - Módulo 4: Algoritmos de ML y Aplicaciones en Ciberseguridad

**Objetivo:** Aplicar algoritmos fundamentales de Machine Learning (Clasificación y Clustering) utilizando la librería Scikit-learn. Practicar el preprocesamiento de datos (escalado, codificación), el entrenamiento de modelos, la predicción y la evaluación de métricas en contextos simplificados de ciberseguridad.

## 1. Preparación: Importar Librerías y Crear Datos de Ejemplo

In [ ]:
# Librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Scikit-learn: Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

# Scikit-learn: Métricas
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, silhouette_score

# Configuraciones opcionales
%matplotlib inline
sns.set_style('whitegrid')

### Dataset 1: Características Simplificadas de URLs (Phishing)

In [ ]:
# Creamos un DataFrame de ejemplo para clasificación de URLs
data_url = {
    'url_length': [15, 65, 25, 80, 40, 95, 30, 110, 20, 55, 70, 85], 
    'num_special_chars': [2, 5, 3, 8, 4, 9, 3, 10, 2, 4, 6, 7],
    'has_ip_address': [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0], # 0: No, 1: Sí
    'num_subdomains': [1, 3, 2, 1, 2, 4, 1, 2, 1, 3, 1, 2],
    'is_phishing': [0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1] # 0: Legítima, 1: Phishing
}
df_url = pd.DataFrame(data_url)

print("Dataset de URLs (Phishing):")
print(df_url.head())

# Separamos features (X) y target (y)
X_url = df_url[['url_length', 'num_special_chars', 'has_ip_address', 'num_subdomains']]
y_url = df_url['is_phishing']

### Dataset 2: Características Simplificadas de Conexiones de Red (Anomalías)

In [ ]:
# Creamos un DataFrame de ejemplo para conexiones de red
data_conn = {
    'duration': [0.1, 0.5, 0.2, 60.0, 0.3, 0.6, 75.0, 0.4, 90.0, 0.2, 0.7, 0.1],
    'num_packets': [10, 50, 20, 5000, 30, 60, 6000, 40, 7500, 25, 70, 15],
    'num_bytes': [1000, 5000, 2000, 500000, 3000, 6000, 600000, 4000, 800000, 2500, 7000, 1500],
    'is_anomaly': [0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0] # 0: Normal, 1: Anomalía
}
df_conn = pd.DataFrame(data_conn)

print("\nDataset de Conexiones de Red (Anomalías):")
print(df_conn.head())

# Features (X) para clasificación y clustering
X_conn = df_conn[['duration', 'num_packets', 'num_bytes']]
# Target (y) solo para clasificación
y_conn = df_conn['is_anomaly']

## 2. Preprocesamiento de Datos con Scikit-learn

**Ejercicio 2.1: Escalado de Características (Scaling)**

* Aplica `StandardScaler` a las características `X_conn` (Dataset 2). Guarda el resultado en `X_conn_scaled_standard`.
* Aplica `MinMaxScaler` a las características `X_conn`. Guarda el resultado en `X_conn_scaled_minmax`.
* Imprime las primeras 5 filas de ambos resultados escalados.
* **Pregunta:** ¿Por qué es importante escalar características para algoritmos como KNN o Regresión Logística (y también para K-Means)?

In [ ]:
# 1. StandardScaler
scaler_standard = StandardScaler()
X_conn_scaled_standard = scaler_standard.fit_transform(X_conn)

print("X_conn escalado con StandardScaler (primeras 5 filas):")
print(X_conn_scaled_standard[:5])

# 2. MinMaxScaler
scaler_minmax = MinMaxScaler()
X_conn_scaled_minmax = scaler_minmax.fit_transform(X_conn)

print("\nX_conn escalado con MinMaxScaler (primeras 5 filas):")
print(X_conn_scaled_minmax[:5])

# 3. Responde a la pregunta en una celda de texto (Markdown) a continuación

**Respuesta a la Pregunta 2.1:**

*Escribe tu respuesta aquí... (Considera cómo afectan las diferentes escalas de las features a los cálculos de distancia o a los coeficientes del modelo)*

**Ejercicio 2.2: Codificación de Variables Categóricas (One-Hot Encoding)**

* Imagina que tenemos una característica 'protocolo' con valores 'TCP', 'UDP', 'ICMP'.
* Crea un pequeño DataFrame de ejemplo con esta columna.
* Aplica `OneHotEncoder` a esta columna.
* Muestra el resultado transformado (será una matriz dispersa o un array denso si usas `sparse_output=False`). ¿Cuántas columnas nuevas se crearon y por qué?

*Nota: `OneHotEncoder` es útil cuando la variable categórica no tiene un orden intrínseco.*

In [ ]:
# DataFrame de ejemplo
df_protocol = pd.DataFrame({'protocolo': ['TCP', 'UDP', 'TCP', 'ICMP', 'UDP']})
print("DataFrame Original:")
print(df_protocol)

# Crear e aplicar OneHotEncoder
# Usamos sparse_output=False para ver un array numpy normal como resultado
encoder = OneHotEncoder(sparse_output=False)
protocol_encoded = encoder.fit_transform(df_protocol[['protocolo']]) # Necesita doble corchete

print("\nResultado del One-Hot Encoding:")
print(protocol_encoded)

print("\nNombres de las nuevas características (columnas):")
print(encoder.get_feature_names_out(['protocolo']))

# Pregunta: ¿Cuántas columnas nuevas se crearon y por qué? (Responde en celda Markdown)

**Respuesta a la Pregunta 2.2:**

*Escribe tu respuesta aquí...*

## 3. Clasificación con Scikit-learn

**Ejercicio 3.1: Regresión Logística para Detección de Phishing**

Pasos:
1.  **Dividir Datos:** Divide `X_url` y `y_url` (Dataset 1) en conjuntos de entrenamiento (80%) y prueba (20%). Usa `random_state=42` para reproducibilidad.
2.  **Preprocesar (Escalar):** Crea un `StandardScaler` y ajústalo **solo** con los datos de entrenamiento (`X_train`). Luego, transforma `X_train` y `X_test`.
3.  **Entrenar Modelo:** Crea una instancia de `LogisticRegression` y entrénala con los datos de entrenamiento escalados (`X_train_scaled`) y las etiquetas de entrenamiento (`y_train`).
4.  **Predecir:** Realiza predicciones sobre el conjunto de prueba escalado (`X_test_scaled`).
5.  **Evaluar:** Calcula Accuracy, Precision, Recall y F1-score comparando las predicciones con `y_test`. Muestra también la Matriz de Confusión.

In [ ]:
# 1. Dividir Datos
X_train_url, X_test_url, y_train_url, y_test_url = train_test_split(
    X_url, y_url, test_size=0.2, random_state=42, stratify=y_url # stratify es bueno para clasificación
)

# 2. Preprocesar (Escalar)
scaler_url = StandardScaler()
X_train_url_scaled = scaler_url.fit_transform(X_train_url)
X_test_url_scaled = scaler_url.transform(X_test_url) # ¡Importante: usar transform, no fit_transform!

# 3. Entrenar Modelo
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_url_scaled, y_train_url)

# 4. Predecir
y_pred_log_reg = log_reg.predict(X_test_url_scaled)

# 5. Evaluar
print("--- Evaluación Regresión Logística (Phishing) ---")
accuracy_lr = accuracy_score(y_test_url, y_pred_log_reg)
precision_lr = precision_score(y_test_url, y_pred_log_reg)
recall_lr = recall_score(y_test_url, y_pred_log_reg)
f1_lr = f1_score(y_test_url, y_pred_log_reg)
cm_lr = confusion_matrix(y_test_url, y_pred_log_reg)

print(f"Accuracy: {accuracy_lr:.4f}")
print(f"Precision: {precision_lr:.4f}")
print(f"Recall: {recall_lr:.4f}")
print(f"F1-score: {f1_lr:.4f}")
print("Matriz de Confusión:")
disp_lr = ConfusionMatrixDisplay(confusion_matrix=cm_lr)
disp_lr.plot()
plt.show()

**Ejercicio 3.2: K-Nearest Neighbors (KNN) para Detección de Phishing**

Repite los pasos del ejercicio 3.1, pero esta vez usando `KNeighborsClassifier`.
1.  Usa los mismos datos divididos y **escalados** (`X_train_url_scaled`, `X_test_url_scaled`, `y_train_url`, `y_test_url`).
2.  Crea una instancia de `KNeighborsClassifier` (puedes empezar con `n_neighbors=3`).
3.  Entrena, predice y evalúa el modelo KNN.
4.  **Pregunta:** Compara brevemente los resultados con la Regresión Logística. ¿Por qué es especialmente importante el escalado para KNN?

In [ ]:
# 1. Usar datos divididos y escalados del ejercicio anterior

# 2. Crear Modelo KNN
knn = KNeighborsClassifier(n_neighbors=3) # Prueba con k=3 inicialmente

# 3. Entrenar, Predecir y Evaluar
knn.fit(X_train_url_scaled, y_train_url)
y_pred_knn = knn.predict(X_test_url_scaled)

print("--- Evaluación KNN (Phishing, k=3) ---")
accuracy_knn = accuracy_score(y_test_url, y_pred_knn)
precision_knn = precision_score(y_test_url, y_pred_knn)
recall_knn = recall_score(y_test_url, y_pred_knn)
f1_knn = f1_score(y_test_url, y_pred_knn)
cm_knn = confusion_matrix(y_test_url, y_pred_knn)

print(f"Accuracy: {accuracy_knn:.4f}")
print(f"Precision: {precision_knn:.4f}")
print(f"Recall: {recall_knn:.4f}")
print(f"F1-score: {f1_knn:.4f}")
print("Matriz de Confusión:")
disp_knn = ConfusionMatrixDisplay(confusion_matrix=cm_knn)
disp_knn.plot()
plt.show()

# 4. Responde a la pregunta en celda Markdown

**Respuesta a la Pregunta 3.2:**

*Compara los resultados... El escalado es importante para KNN porque...*

**Ejercicio 3.3: Random Forest para Detección de Phishing**

Ahora, usa un `RandomForestClassifier`.
1.  Usa los mismos datos divididos (`X_train_url`, `X_test_url`, `y_train_url`, `y_test_url`). Nota: Los árboles y bosques **no siempre requieren escalado**, así que puedes probar a usar los datos *sin escalar* esta vez para comparar (o usar los escalados si prefieres).
2.  Crea una instancia de `RandomForestClassifier` (puedes usar `n_estimators=100`, `random_state=42`).
3.  Entrena, predice y evalúa el modelo Random Forest.
4.  Compara los resultados con los modelos anteriores.

In [ ]:
# 1. Usar datos divididos (prueba con/sin escalar, aquí usamos sin escalar)

# 2. Crear Modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 3. Entrenar, Predecir y Evaluar (Usando datos NO escalados)
rf.fit(X_train_url, y_train_url) # Entrenar con datos originales
y_pred_rf = rf.predict(X_test_url)   # Predecir con datos originales

print("--- Evaluación Random Forest (Phishing) ---")
accuracy_rf = accuracy_score(y_test_url, y_pred_rf)
precision_rf = precision_score(y_test_url, y_pred_rf)
recall_rf = recall_score(y_test_url, y_pred_rf)
f1_rf = f1_score(y_test_url, y_pred_rf)
cm_rf = confusion_matrix(y_test_url, y_pred_rf)

print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-score: {f1_rf:.4f}")
print("Matriz de Confusión:")
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf)
disp_rf.plot()
plt.show()

# 4. Compara mentalmente o añade una celda Markdown para comparar

## 4. Clustering con Scikit-learn

**Ejercicio 4.1: K-Means para Agrupar Conexiones de Red**

Usaremos `X_conn` (Dataset 2) *sin* las etiquetas `y_conn` para ver si K-Means puede agrupar conexiones similares (potencialmente separando normales de anómalas).

1.  **Preprocesar (Escalar):** Es **muy importante** escalar los datos para K-Means. Usa el `X_conn_scaled_standard` (o `_minmax`) que calculaste en el Ejercicio 2.1.
2.  **Entrenar Modelo:** Crea una instancia de `KMeans` con `n_clusters=2` (ya que sabemos que hay 2 tipos: normal/anomalía, aunque K-Means no lo sabe). Usa `random_state=42` y `n_init='auto'`.
3.  Entrena el modelo con los datos escalados (`kmeans.fit()`).
4.  **Obtener Etiquetas:** Obtén las etiquetas de cluster asignadas a cada punto (`kmeans.labels_`).
5.  **Visualizar:** Crea un gráfico de dispersión (scatter plot) usando dos de las características originales (ej. 'duration' vs 'num_packets' de `X_conn`) y colorea los puntos según las etiquetas de cluster encontradas por K-Means. Añade los centroides al gráfico (`kmeans.cluster_centers_`, ¡pero recuerda que están en la escala transformada! Necesitarías `scaler.inverse_transform()` para ponerlos en la escala original si quieres superponerlos directamente al gráfico de datos originales).
6.  **Pregunta:** Observando el gráfico y las características de los datos originales (Dataset 2), ¿parece que K-Means ha logrado separar (al menos parcialmente) las conexiones normales de las anómalas? ¿Cómo podría usarse esto para detectar anomalías?

In [ ]:
# 1. Usar datos escalados (X_conn_scaled_standard del Ejercicio 2.1)
# Asegúrate de que X_conn_scaled_standard esté disponible
if 'X_conn_scaled_standard' not in locals():
    print("Ejecuta primero la celda del Ejercicio 2.1 para escalar X_conn")
    # O vuelve a calcularlo aquí:
    # scaler_standard = StandardScaler()
    # X_conn_scaled_standard = scaler_standard.fit_transform(X_conn)

# 2. Crear Modelo K-Means
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')

# 3. Entrenar Modelo
kmeans.fit(X_conn_scaled_standard)

# 4. Obtener Etiquetas
cluster_labels = kmeans.labels_
print("Etiquetas de Cluster asignadas:", cluster_labels)

# 5. Visualizar (usando datos originales para los ejes, coloreados por cluster)
plt.figure(figsize=(8, 6))
scatter = sns.scatterplot(x=X_conn['duration'], 
                          y=X_conn['num_packets'], 
                          hue=cluster_labels, 
                          palette='viridis', 
                          s=100, # Tamaño de los puntos
                          alpha=0.7)

plt.title('Clusters de Conexiones de Red encontrados por K-Means (k=2)')
plt.xlabel('Duración')
plt.ylabel('Número de Paquetes')
plt.legend(title='Cluster ID')
plt.show()

# Opcional: Calcular Silhouette Score (una métrica de clustering)
try:
    silhouette_avg = silhouette_score(X_conn_scaled_standard, cluster_labels)
    print(f"\nSilhouette Score: {silhouette_avg:.4f}") 
    # Más cerca de 1 es mejor, más cerca de -1 es peor.
except NameError: # En caso de que X_conn_scaled_standard no se haya creado
    print("\nNo se pudo calcular Silhouette Score porque los datos escalados no están definidos.")

# 6. Responde a la pregunta en celda Markdown

**Respuesta a la Pregunta 4.1:**

*Observando el gráfico... ¿Separó los puntos con valores altos de duration/packets? Esto podría usarse para detectar anomalías si...*

## 5. Ingeniería de Características (Ejercicio Conceptual)

**Ejercicio 5.1: Creación de Features desde Logs**

Imagina que tienes logs de firewall con líneas como esta:

`Apr 15 20:45:10 firewall1 DENY TCP src=10.1.1.10 dst=192.168.1.5 sport=54321 dport=80 len=60`

Si quisieras usar Machine Learning para analizar estos logs (por ejemplo, para detectar patrones de ataque):

1.  Enumera al menos 5 **features** (características) que podrías extraer de una línea de log como esta.
2.  Indica si cada feature que has enumerado sería probablemente **numérica** o **categórica**.

**Respuesta a la Pregunta 5.1:**

1.  *Feature 1: ... (Tipo: ...)*
2.  *Feature 2: ... (Tipo: ...)*
3.  *Feature 3: ... (Tipo: ...)*
4.  *Feature 4: ... (Tipo: ...)*
5.  *Feature 5: ... (Tipo: ...)*
    *(Ejemplos: hora del día, día de la semana, acción (DENY/ALLOW), protocolo (TCP/UDP), IP origen/destino (podrían ser categóricas o procesadas), puerto origen/destino, longitud del paquete)*

--- 
¡Gran trabajo! Has aplicado varios algoritmos de ML usando Scikit-learn, incluyendo pasos cruciales como el preprocesamiento y la evaluación. Ahora estás mucho más cerca de poder aplicar estas técnicas a problemas reales de ciberseguridad, como los que se exploran en el libro de referencia.